# Linear Regression in Python

Linear regression in Python is available through the *sklearn* machine learning library. In this hands-on exercise we will examine the relationship between various attributes of blocks of households in California and the median house value for California districts. To begin let's import some libraries:

In [18]:
pip list

Package                   Version
------------------------- ---------------
anyio                     4.0.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.2.3
asttokens                 2.4.0
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.12.1
backcall                  0.2.0
bcrypt                    4.0.1
beautifulsoup4            4.12.2
bleach                    6.0.0
blinker                   1.4
capstone                  4.0.2
certifi                   2022.12.7
cffi                      1.15.1
charset-normalizer        3.1.0
cmake                     3.26.3
colored-traceback         0.3.0
comm                      0.1.4
command-not-found         0.3
cryptography              40.0.1
dbus-python               1.2.18
debugpy                   1.7.0
decorator                 5.1.1
defusedxml                0.7.1
distlib                   0.3.4
distro                    1.7.0
distro-info        

In [13]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn

In [ ]:
The numpy library provides us with some convenient ways to represent and manipulate vectors and matrices, which we will use extensively in this course. 

The pandas library provides very nice tools for managing datasets, while scipy is the Python scientific library with useful functions. 

The main star of this show is sklearn, which is the Python machine learning library, providing us with tools to create regression models using linear regression, and classification models using many models including Bayesian models and Support Vector Machines, which we will look at in this lecture.

In the lab we will see how to use our own datasets, but for this lecture we will use some toy datasets that are given in sklearn. The toy datasets, amongst many others, include:

- iris: Dataset on iris flowers for classification.
- diabetes: A diabetes regression dataset.
- digits: A collection of 8x8 images of digits for classification.
- wines: Data on wine types for classification.
- linnerrud: Dataset of physiological features of members of a fitness club for regression.
- breast cancer: Breast cancer dataset for regression.

Feel free to play around with the above datasets.

## Loading the California housing dataset
https://scikit-learn.org/stable/datasets

For now we will use the California housing dataset. To load it:


In [26]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()

We can see how big out dataset is:

In [27]:
housing.data.shape

(20640, 8)

From here we see that there's 20640 pieces of data, with 8 attributes each. Let's look at what's in this dataset by printing the first 5 entries:

In [28]:
np.set_printoptions(precision = 3, suppress = True)
d = repr(housing.data[:5]) # Nice printable version of housing.data
print(d)

array([[   8.325,   41.   ,    6.984,    1.024,  322.   ,    2.556,
          37.88 , -122.23 ],
       [   8.301,   21.   ,    6.238,    0.972, 2401.   ,    2.11 ,
          37.86 , -122.22 ],
       [   7.257,   52.   ,    8.288,    1.073,  496.   ,    2.802,
          37.85 , -122.24 ],
       [   5.643,   52.   ,    5.817,    1.073,  558.   ,    2.548,
          37.85 , -122.25 ],
       [   3.846,   52.   ,    6.282,    1.081,  565.   ,    2.181,
          37.85 , -122.25 ]])


As we can see we have a Numpy array of 8 attributes, just as we found when we looked at the shape of the array. To see what each attribute does:

In [29]:
print(housing.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

This gives us a very nice description of the 8 attributes that are available for regression. There is a 9th attribute (the target) which is the median house value for California districts, expressed in hundreds of thousands of dollars

## Creating a Pandas Dataframe
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

To make it easier to manipulate the data, let's convert it to a Pandas dataframe, and print the first five rows:

In [30]:
cal = pd.DataFrame(housing.data)
cal.head()

,0,1,2,3,4,5,6,7
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


Very nice, but the columns don't have headings. This is set using the list cal.columns, and as it turns out the headings are available as a list in housing.feature_names. We can just set the cal.columns accordingly. You can also load just reload the dataset as a dataframe using  fetch_california_housing(as_frame = True).

In [31]:
cal.columns = housing.feature_names
cal.head()

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25


We can now explore correlations between various columns of our data. For example, we may expect that a house with more rooms should have more bedrooms, or that the higher the median income of an area, the larger the house (i.e. more rooms). We make use of correlation to test these ideas. 

A correlation of 1 between two factors A and B means that factor A depends perfectly on factor B, while a correlation of -1 means that factor A is perfectly negatively correlated with factor B. We can find correlation using:

```
dataframe['A'].corr(dataframe['B'])
```


In [32]:
room_bedroom = cal['AveBedrms'].corr(cal['AveRooms'])
age_pop = cal['HouseAge'].corr(cal['Population'])
inc_room = cal['MedInc'].corr(cal['AveRooms'])


print("corr rooms/bedrooms: %3.3f, corr house_age/population: %3.3f, corr median_income/rooms: %3.3f" 
      % (room_bedroom, age_pop, inc_room))


corr rooms/bedrooms: 0.848, corr house_age/population: -0.296, corr median_income/rooms: 0.327


As expected, we see that the number of rooms and bedrooms have a strong correlation. We also see that there is some correlation between median icome and the average number of rooms. What is interesting is that population has an inverse correlation with house age. This is something worth looking at. :) But that's not why we are here.

## Finding the Relationship Between Property Values and Median Income

Let's see how we can create a linear regression model to find the relationship between property values and median income. In the  dataset, we can find the property values in housing.target. We first create a new column called "Value" in the Pandas dataframe, and then check that there is actually a correlation between property values and median income.

In [33]:
cal['Value'] = housing.target
print("Correlation between property values and median income: %3.3f"
     % cal['Value'].corr(cal['MedInc']))

Correlation between property values and median income: 0.688


Here we can see a moderate correlation between property values and median income, as we would expect. Let's start building our training data from the Pandas dataframe:

### Creating the Data

We will create the training data by extracting the target or "dependent variable" (property values) and the data driving the target (the "independent variable"). We then use linear regression to find the model relating independent and dependent variables.  We begin by converting both variables to from row vectors to column vectors:

In [35]:
X = cal['MedInc'].values.reshape(-1, 1)
Y = cal['Value'].values.reshape(-1, 1)

### Splitting into Training and Testing Data
https://scikit-learn.org/stable/model_selection.html

Before we create a model we always want to split the data into training data and testing (or validation) data. This allows us to test the model's "goodness of fit" against data it has seen for training, and data it has never seen before ('unseen data'). This is to test for "overfitting", where the model memorizes the training data and has excellent result, but produces very poor results for unseen data. We want to ensure that the results for both training data and testing data do not vary too greatly.

We will use the train_test_split function in sklearn to put aside 33% of the data for testing. 

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33)


We will verify that we indeed have 33% of our data for testing:

In [37]:
all_rows = X.shape[0]
test_rows = X_test.shape[0]
print("%%age of data used for test: %3.2f%%" % (test_rows / all_rows * 100.0))

%age of data used for test: 33.00%


### Creating the Regression Model

Excellent! Now let's build our regression model. We start by importing the LinearRegression class from sklearn:

In [38]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

Now we can use the fit method to learn from our data, and look at the coefficient and intercept for our model:


In [39]:
lm.fit(X_train, Y_train)
print("Coefficient: %3.4f, Intercept: %3.4f." % (lm.coef_[0][0], lm.intercept_[0]))

Coefficient: 0.4226, Intercept: 0.4373.


The coefficient tells us how property values increase (since it is positive) for each increase in median income, while the intercept is a base value at $0 median income.  This gives us some insight into how median income effects property values (or maybe how property values effect median income). Note that you might get slightly different values here.

### Evaluating for Over-fitting

We briefly mentioned overfitting earlier in this document; a model has "overfitted" its training data if it can perform very well predicting the outputs for training data, yet perform very poorly on data it has never seen before.

So now lets now look at how well this model fits the training data and the testing data. We will take the root-mean-square error (RMSE) given by:

$$
RMSE = \sqrt{\frac{\sum_{i=0}^{n-1}(y_{a,i} - y_{m,i})^2}{n} }
$$

Here $y_{a,i}$ is the i-th *actual* value from the data, and $y_{m, i}$ is the corresponding output from the model.

On its own the RMSE is fairly useless, but we can use it to compare the prediction error of the model when it is using training data it has seen, and testing data it has never seen.

We call lm.predict(X_train) and lm.predict(X_test) to produce the predicted property values using the training and testing data respectively, then call the mean_squared_error function from the sklearn.metrics package and square-root the result. Recall that our target values are in Y_train and Y_test:

In [40]:
from sklearn import metrics
Y_pred_train = lm.predict(X_train)
Y_pred_test = lm.predict(X_test)
train_mse = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
test_mse = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
print("RMSE for training data: %3.4f. RMSE for testing data: %3.4f." % (train_mse, test_mse))
                

RMSE for training data: 0.8362. RMSE for testing data: 0.8398.


On its own the RMSE is fairly close. This means that the model performs about as well on data it has never seen, as on training data it has seen. So we are confident that the model is sufficiently "general" and has not memorized the training data to the detriment of unseen data.

Note however that the RMSE does not give us a really good idea of how accurate our model is in absolute terms, just in relative terms. We can still use this however to compare against other models we create.



### Playing with Our Model

We can now make some predictions on our model. Note that we are not given the units of the median income, so we will assume it is in units of \\$10K here. We look at property values if the median income was \\$10K (in units of \\$10,000), versus values when the median income was \\$100K (in units of \\$10,000). We note that the values are in units of $100,000 and multiply accordingly.

Note that lm.predict requires a 2D numpy array, which we create using np.array.

In [41]:
value1 = lm.predict(np.array([[1]]))
value2 = lm.predict(np.array([[10.0]]))

print("Value at a median income of $10K is $%3.2f. Value at a median income of $100K is $%3.2f"
      % (value1[0][0] * 100000, value2[0][0] * 100000))


Value at a median income of $10K is $85987.32. Value at a median income of $100K is $466346.08


From here we see that at a median income of \\$10K the house value is \\$85,587.68, which increases to \\$463,792.44 when the median income increases to \\$100K. We can manually verify that this is true:

In [42]:
v1 = 0.4209 * 1 + 0.4398
v2 = 0.4209 * 10 + 0.4398

print("Value at a median income of $10K is $%3.2f. Value at a median income of $100K is $%3.2f"
      % (v1 * 100000, v2 * 100000))

Value at a median income of $10K is $86070.00. Value at a median income of $100K is $464880.00


We see an almost perfect match between the two models. :)